In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/프로젝트 폴더/data/selected_data

/content/drive/.shortcut-targets-by-id/1fmVXXO9WkxfQqimV76GhZhDAdNd3hhtJ/프로젝트 폴더/data/selected_data


In [3]:
%ls

data_out.csv   강남구예상매출.csv  상권영역.csv      예상매출.csv
merged.csv     매출비교.csv        상권점포.csv      행정동코드.csv
model_reg.pkl  매출비교.gsheet     상권직장인구.csv
model_r.pkl    상권상주인구.csv    상권집객시설.csv
result.csv     상권생활인구.csv    상권추정매출.csv


In [4]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings(action='ignore')

#### 데이터 불러오기

In [5]:
상권상주인구 = pd.read_csv('상권상주인구.csv')
상권직장인구 = pd.read_csv('상권직장인구.csv')
상권생활인구 = pd.read_csv('상권생활인구.csv')
상권추정매출 = pd.read_csv('상권추정매출.csv')
상권집객시설 = pd.read_csv('상권집객시설.csv')
행정동코드 = pd.read_csv('행정동코드.csv')     # 상권코드 - 행정동 코드 매칭
상권영역 = pd.read_csv('상권영역.csv')         # 행정동코드 - 자치구 코드 매칭

In [6]:
# 한글 포함된 컬럼들 모두 삭제
상권상주인구.drop(columns='상권코드명',inplace=True)  #
상권직장인구.drop(columns='상권코드명',inplace=True)
상권생활인구.drop(columns='상권코드명',inplace=True)  #
상권추정매출.drop(columns='상권코드명',inplace=True)  #
상권집객시설.drop(columns='상권코드명',inplace=True)

In [7]:
'''
CS100001 : 한식음식점
CS100002 : 중식음식점
CS100003 : 일식음식점
CS100004 : 양식음식점
CS100005 : 제과점
CS100006 : 패스트푸드점점
CS100007 : 치킨전문점
CS100008 : 분식전문점
CS100009 : 호프-간이주점
CS100010 : 커피-음료
CS300007 : 육류판매
'''

'\nCS100001 : 한식음식점\nCS100002 : 중식음식점\nCS100003 : 일식음식점\nCS100004 : 양식음식점\nCS100005 : 제과점\nCS100006 : 패스트푸드점점\nCS100007 : 치킨전문점\nCS100008 : 분식전문점\nCS100009 : 호프-간이주점\nCS100010 : 커피-음료\nCS300007 : 육류판매\n'

In [8]:
업종 = 'CS100001|CS100002|CS100003|CS100004|CS100005|CS100006|CS100007|CS100008|CS100009|CS100010|CS300007'

#### 상권추정매출 정리

In [9]:
상권추정매출 = 상권추정매출[상권추정매출['서비스업종코드'].str.contains(업종, na = False)]

In [10]:
상권추정매출.drop(columns='서비스업종코드',inplace=True)

In [11]:
상권추정매출.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250427 entries, 4 to 928238
Data columns (total 52 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   기준년코드            250427 non-null  int64  
 1   기준분기코드           250427 non-null  int64  
 2   상권코드             250427 non-null  int64  
 3   서비스업종코드명         250427 non-null  object 
 4   분기당매출금액          250427 non-null  float64
 5   분기당_매출_건수        250427 non-null  int64  
 6   주중_매출_비율         250427 non-null  int64  
 7   주말_매출_비율         250427 non-null  int64  
 8   월요일_매출_비율        250427 non-null  int64  
 9   화요일_매출_비율        250427 non-null  int64  
 10  수요일_매출_비율        250427 non-null  int64  
 11  목요일_매출_비율        250427 non-null  int64  
 12  금요일_매출_비율        250427 non-null  int64  
 13  토요일_매출_비율        250427 non-null  int64  
 14  일요일_매출_비율        250427 non-null  int64  
 15  시간대_00~06_매출_비율  250427 non-null  int64  
 16  시간대_06~11_매출_비율  250427 non-null  int6

In [12]:
상권추정매출 = 상권추정매출.replace('*','1')

In [13]:
상권추정매출 = 상권추정매출.astype({'점포수': 'int'})

In [14]:
상권추정매출['분기당매출금액'] = 상권추정매출['분기당매출금액']/상권추정매출['점포수']/3      # 분기당매출금액을 업종별 월매출로 변환(컬럼명은 그대로 사용)

In [15]:
상권추정매출 = 상권추정매출[['기준년코드','기준분기코드','상권코드','서비스업종코드명','분기당매출금액']]

In [16]:
상권추정매출['분기당매출금액'] = np.round(상권추정매출['분기당매출금액'],-4)

In [17]:
상권추정매출.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250427 entries, 4 to 928238
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   기준년코드     250427 non-null  int64  
 1   기준분기코드    250427 non-null  int64  
 2   상권코드      250427 non-null  int64  
 3   서비스업종코드명  250427 non-null  object 
 4   분기당매출금액   250427 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 11.5+ MB


In [18]:
상권추정매출.describe()

,기준년코드,기준분기코드,상권코드,분기당매출금액
count,250427.000000,250427.000000,2.504270e+05,2.504270e+05
mean,2019.399098,2.493665,2.109242e+06,1.141145e+07
std,1.713516,1.118331,8.613342e+04,2.285272e+07
min,2017.000000,1.000000,1.001491e+06,0.000000e+00
25%,2018.000000,1.000000,2.110437e+06,3.130000e+06
50%,2019.000000,2.000000,2.110871e+06,6.600000e+06
75%,2021.000000,3.000000,2.120153e+06,1.298000e+07
max,2022.000000,4.000000,2.130326e+06,2.255670e+09


#### 상권생활인구 정리

In [19]:
col_상권생활인구 = ['기준년코드','기준분기코드','상권코드','총_생활인구_수','남성_생활인구_수','여성_생활인구_수','연령대_10_생활인구_수','연령대_20_생활인구_수','연령대_30_생활인구_수','연령대_40_생활인구_수','연령대_50_생활인구_수','연령대_60_이상_생활인구_수','시간대_1_생활인구_수','시간대_2_생활인구_수','시간대_3_생활인구_수','시간대_4_생활인구_수','시간대_5_생활인구_수','시간대_6_생활인구_수','월요일_생활인구_수','화요일_생활인구_수','수요일_생활인구_수','목요일_생활인구_수','금요일_생활인구_수','토요일_생활인구_수','일요일_생활인구_수']
상권생활인구 = 상권생활인구[col_상권생활인구]

In [20]:
상권생활인구.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40041 entries, 0 to 40040
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   기준년코드             40041 non-null  int64
 1   기준분기코드            40041 non-null  int64
 2   상권코드              40041 non-null  int64
 3   총_생활인구_수          40041 non-null  int64
 4   남성_생활인구_수         40041 non-null  int64
 5   여성_생활인구_수         40041 non-null  int64
 6   연령대_10_생활인구_수     40041 non-null  int64
 7   연령대_20_생활인구_수     40041 non-null  int64
 8   연령대_30_생활인구_수     40041 non-null  int64
 9   연령대_40_생활인구_수     40041 non-null  int64
 10  연령대_50_생활인구_수     40041 non-null  int64
 11  연령대_60_이상_생활인구_수  40041 non-null  int64
 12  시간대_1_생활인구_수      40041 non-null  int64
 13  시간대_2_생활인구_수      40041 non-null  int64
 14  시간대_3_생활인구_수      40041 non-null  int64
 15  시간대_4_생활인구_수      40041 non-null  int64
 16  시간대_5_생활인구_수      40041 non-null  int64
 17  시간대_6_생활인구_수      40041 non-nul

#### 상권상주인구 정리

In [21]:
상권상주인구.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39178 entries, 0 to 39177
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준년코드   39178 non-null  int64  
 1   기준분기코드  39178 non-null  int64  
 2   상권코드    39178 non-null  int64  
 3   총상주인구수  39178 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 1.2 MB


#### 상권직장인구 정리

In [22]:
상권직장인구.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37744 entries, 0 to 37743
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준년코드   37744 non-null  int64  
 1   기준분기코드  37744 non-null  int64  
 2   상권코드    37744 non-null  int64  
 3   총직장인구수  37744 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 1.2 MB


#### 상권집객시설

In [23]:
상권집객시설.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38022 entries, 0 to 38021
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준년코드   38022 non-null  int64  
 1   기준분기코드  38022 non-null  int64  
 2   상권코드    38022 non-null  int64  
 3   집객시설수   38022 non-null  float64
 4   관공서수    38022 non-null  float64
 5   은행수     38022 non-null  float64
 6   버스정거장수  38022 non-null  float64
 7   지하철수    38022 non-null  float64
 8   약국수     38022 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 2.6 MB


#### 상권점포

In [24]:
상권점포 = pd.read_csv('상권점포.csv')

In [25]:
상권점포.drop(columns='상권코드명',inplace=True)

In [26]:
상권점포 = 상권점포[상권점포['서비스업종코드'].str.contains(업종, na = False)]

In [27]:
상권점포.drop(columns='서비스업종코드',inplace=True)

In [28]:
상권점포.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320081 entries, 19 to 1787961
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   기준년코드     320081 non-null  int64  
 1   기준분기코드    320081 non-null  int64  
 2   상권코드      320081 non-null  int64  
 3   서비스업종코드명  320081 non-null  object 
 4   점포수       320081 non-null  float64
 5   유사업종점포수   320081 non-null  float64
 6   개업율       320081 non-null  float64
 7   개업점포수     320081 non-null  float64
 8   폐업율       320081 non-null  float64
 9   폐업점포수     320081 non-null  float64
dtypes: float64(6), int64(3), object(1)
memory usage: 26.9+ MB


#### 하나의 데이터프레임으로 병합 -> 일단 가장 row수가 많은 걸 기준으로 join

In [29]:
print('상권점포 :',len(상권점포))
print('상권집객시설 :',len(상권집객시설))
print('상권직장인구 :',len(상권직장인구))
print('상권상주인구 :',len(상권상주인구))
print('상권생활인구 :',len(상권생활인구))
print('상권추정매출 :',len(상권추정매출))

상권점포 : 320081
상권집객시설 : 38022
상권직장인구 : 37744
상권상주인구 : 39178
상권생활인구 : 40041
상권추정매출 : 250427


In [30]:
merged_df = pd.merge(상권점포,상권집객시설, on = ['기준년코드', '기준분기코드', '상권코드'], how='left')

In [31]:
merged_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320081 entries, 0 to 320080
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   기준년코드     320081 non-null  int64  
 1   기준분기코드    320081 non-null  int64  
 2   상권코드      320081 non-null  int64  
 3   서비스업종코드명  320081 non-null  object 
 4   점포수       320081 non-null  float64
 5   유사업종점포수   320081 non-null  float64
 6   개업율       320081 non-null  float64
 7   개업점포수     320081 non-null  float64
 8   폐업율       320081 non-null  float64
 9   폐업점포수     320081 non-null  float64
 10  집객시설수     311795 non-null  float64
 11  관공서수      311795 non-null  float64
 12  은행수       311795 non-null  float64
 13  버스정거장수    311795 non-null  float64
 14  지하철수      311795 non-null  float64
 15  약국수       311795 non-null  float64
dtypes: float64(12), int64(3), object(1)
memory usage: 41.5+ MB


In [32]:
merged_df = pd.merge(merged_df,상권직장인구, on = ['기준년코드', '기준분기코드', '상권코드'])

In [33]:
merged_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304814 entries, 0 to 304813
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   기준년코드     304814 non-null  int64  
 1   기준분기코드    304814 non-null  int64  
 2   상권코드      304814 non-null  int64  
 3   서비스업종코드명  304814 non-null  object 
 4   점포수       304814 non-null  float64
 5   유사업종점포수   304814 non-null  float64
 6   개업율       304814 non-null  float64
 7   개업점포수     304814 non-null  float64
 8   폐업율       304814 non-null  float64
 9   폐업점포수     304814 non-null  float64
 10  집객시설수     297564 non-null  float64
 11  관공서수      297564 non-null  float64
 12  은행수       297564 non-null  float64
 13  버스정거장수    297564 non-null  float64
 14  지하철수      297564 non-null  float64
 15  약국수       297564 non-null  float64
 16  총직장인구수    304814 non-null  float64
dtypes: float64(13), int64(3), object(1)
memory usage: 41.9+ MB


In [34]:
merged_df = pd.merge(merged_df,상권상주인구, on = ['기준년코드', '기준분기코드', '상권코드'])

In [35]:
merged_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302320 entries, 0 to 302319
Data columns (total 18 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   기준년코드     302320 non-null  int64  
 1   기준분기코드    302320 non-null  int64  
 2   상권코드      302320 non-null  int64  
 3   서비스업종코드명  302320 non-null  object 
 4   점포수       302320 non-null  float64
 5   유사업종점포수   302320 non-null  float64
 6   개업율       302320 non-null  float64
 7   개업점포수     302320 non-null  float64
 8   폐업율       302320 non-null  float64
 9   폐업점포수     302320 non-null  float64
 10  집객시설수     295426 non-null  float64
 11  관공서수      295426 non-null  float64
 12  은행수       295426 non-null  float64
 13  버스정거장수    295426 non-null  float64
 14  지하철수      295426 non-null  float64
 15  약국수       295426 non-null  float64
 16  총직장인구수    302320 non-null  float64
 17  총상주인구수    302320 non-null  float64
dtypes: float64(14), int64(3), object(1)
memory usage: 43.8+ MB


In [36]:
merged_df = pd.merge(merged_df,상권생활인구, on = ['기준년코드', '기준분기코드', '상권코드'])

In [37]:
merged_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302320 entries, 0 to 302319
Data columns (total 40 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   기준년코드             302320 non-null  int64  
 1   기준분기코드            302320 non-null  int64  
 2   상권코드              302320 non-null  int64  
 3   서비스업종코드명          302320 non-null  object 
 4   점포수               302320 non-null  float64
 5   유사업종점포수           302320 non-null  float64
 6   개업율               302320 non-null  float64
 7   개업점포수             302320 non-null  float64
 8   폐업율               302320 non-null  float64
 9   폐업점포수             302320 non-null  float64
 10  집객시설수             295426 non-null  float64
 11  관공서수              295426 non-null  float64
 12  은행수               295426 non-null  float64
 13  버스정거장수            295426 non-null  float64
 14  지하철수              295426 non-null  float64
 15  약국수               295426 non-null  float64
 16  총직장인구수            30

In [38]:
merged_df = pd.merge(merged_df,상권추정매출 , on = ['기준년코드', '기준분기코드', '상권코드','서비스업종코드명'])

In [39]:
merged_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234091 entries, 0 to 234090
Data columns (total 41 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   기준년코드             234091 non-null  int64  
 1   기준분기코드            234091 non-null  int64  
 2   상권코드              234091 non-null  int64  
 3   서비스업종코드명          234091 non-null  object 
 4   점포수               234091 non-null  float64
 5   유사업종점포수           234091 non-null  float64
 6   개업율               234091 non-null  float64
 7   개업점포수             234091 non-null  float64
 8   폐업율               234091 non-null  float64
 9   폐업점포수             234091 non-null  float64
 10  집객시설수             230165 non-null  float64
 11  관공서수              230165 non-null  float64
 12  은행수               230165 non-null  float64
 13  버스정거장수            230165 non-null  float64
 14  지하철수              230165 non-null  float64
 15  약국수               230165 non-null  float64
 16  총직장인구수            23

In [40]:
merged_df.head()

,기준년코드,기준분기코드,상권코드,서비스업종코드명,점포수,유사업종점포수,개업율,개업점포수,폐업율,폐업점포수,...,시간대_5_생활인구_수,시간대_6_생활인구_수,월요일_생활인구_수,화요일_생활인구_수,수요일_생활인구_수,목요일_생활인구_수,금요일_생활인구_수,토요일_생활인구_수,일요일_생활인구_수,분기당매출금액
0,2022,2,2110308,호프-간이주점,4.0,4.0,0.0,0.0,0.0,0.0,...,140507,103437,109943,108580,109076,109850,108050,123873,125029,2200000.0
1,2022,2,2110308,한식음식점,9.0,9.0,0.0,0.0,11.0,1.0,...,140507,103437,109943,108580,109076,109850,108050,123873,125029,9620000.0
2,2022,2,2110308,치킨전문점,0.0,2.0,0.0,0.0,0.0,0.0,...,140507,103437,109943,108580,109076,109850,108050,123873,125029,23910000.0
3,2022,2,2110308,패스트푸드점,0.0,2.0,0.0,0.0,0.0,0.0,...,140507,103437,109943,108580,109076,109850,108050,123873,125029,16730000.0
4,2022,2,2110308,분식전문점,6.0,6.0,17.0,1.0,0.0,0.0,...,140507,103437,109943,108580,109076,109850,108050,123873,125029,5690000.0


In [41]:
상권영역.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   행정동코드   1671 non-null   int64 
 1   시군구코드   1671 non-null   int64 
 2   XCNTS   1671 non-null   int64 
 3   YDNTS   1671 non-null   int64 
 4   상권코드    1671 non-null   int64 
 5   상권코드명   1671 non-null   object
dtypes: int64(5), object(1)
memory usage: 78.5+ KB


In [42]:
%cd /content/drive/MyDrive/프로젝트 폴더/code/Sales_model/Data

/content/drive/.shortcut-targets-by-id/1fmVXXO9WkxfQqimV76GhZhDAdNd3hhtJ/프로젝트 폴더/code/Sales_model/Data


In [43]:
merged_df.to_csv('merged.csv', encoding='utf-8-sig')